# Intelligent Agents: Reflex Agents for the Vacuum-cleaner World


## Instructions

Total Points: undergrad 10, graduate students 11

Complete this notebook and submit it. The notebook needs to be a complete project report with 

* your implementation,
* documentation including a short discussion of how your implementation works and your design choices, and
* experimental results (e.g., tables and charts with simulation results) with a short discussion of what they mean. 

Use the provided notebook cells and insert additional code and markdown cells as needed.

## Introduction

In this assignment you will implement a simulator environment for an automatic vacuum cleaner robot, a set of different reflex agent programs, and perform a comparison study for cleaning a single room. Focus on the __cleaning phase__ which starts when the robot is activated and ends when the last dirty square is cleaned. Someone else will take care of the agent program needed to navigate back to the charging station after the room is clean.

## PEAS description of the cleaning phase

__Performance Measure:__ Each action costs 1 energy unit. The performance is measured as the sum of the energy units used to clean the whole room.

__Environment:__ A room with $n \times n$ squares where $n = 5$. Dirt is randomly placed on each square with probability $p = 0.2$. For simplicity, you can assume that the agent knows the size and the layout of the room (i.e., it knows $n$). To start, the agent is placed on a random square.

__Actuators:__ The agent can `clean` the current square or move to an adjacent square by going `north`, `east`, `south`, or `west`.

__Sensors:__ Four bumper sensors, one for north, east, south, and west; a dirt sensor reporting dirt in the current square.  


## The agent program for a simple randomized agent

The agent program is a function that gets sensor information (the current percepts) as the arguments. The arguments are:

* A dictionary with boolean entries for the for bumper sensors `north`, `east`, `west`, `south`. E.g., if the agent is on the north-west corner, `bumpers` will be `{"north" : True, "east" : False, "south" : False, "west" : True}`.
* The dirt sensor produces a boolean.

The agent returns the chosen action as a string.

Here is an example implementation for the agent program of a simple randomized agent:  

In [1]:
from numpy import random

actions = ["north", "east", "west", "south", "suck"]

def simple_randomized_agent(bumpers, dirty):
    return random.choice(actions)

In [2]:
# define percepts (current location is NW corner and it is dirty)
bumpers = {"north" : True, "east" : False, "south" : False, "west" : True}
dirty = True

# call agent program function with percepts and it returns an action
simple_randomized_agent(bumpers, dirty)

'west'

__Note:__ This is not a rational intelligent agent. It ignores its sensors and may bump into a wall or not clean a dirty square. You will be asked to implement rational agents below.

## Simple environment example

This simple environment is infinite in size (bumpers are always `False`) and every square is always dirty, even if the agent cleans it. The environment function returns the performance measure which is here the number of cleaned squares (since all squares are constantly dirty, it is the number of `suck` actions by the agent). 

In [3]:
def simple_environment(agent, max_steps, verbose = True):
    num_cleaned = 0
    
    for i in range(max_steps):
        dirty = True
        bumpers = {"north" : False, "south" : False, "west" : False, "east" : False}

        action = agent(bumpers, dirty)
        if (verbose): print("step", i , "- action:", action) 
        
        if (action == "suck"): 
            num_cleaned = num_cleaned + 1
        
    return num_cleaned
        


Do one simulation run with 20 steps.

In [4]:
simple_environment(simple_randomized_agent, max_steps = 20)

step 0 - action: west
step 1 - action: north
step 2 - action: suck
step 3 - action: west
step 4 - action: east
step 5 - action: north
step 6 - action: east
step 7 - action: east
step 8 - action: west
step 9 - action: suck
step 10 - action: east
step 11 - action: west
step 12 - action: east
step 13 - action: south
step 14 - action: south
step 15 - action: suck
step 16 - action: north
step 17 - action: west
step 18 - action: east
step 19 - action: suck


4

# Tasks

_Submission Instructions:_ Use this notebook to prepare your submission. Complete this section with your code and results. You can add additional Markdown blocks for your description, comments in the code and use mathplotlib to produce charts. 

_Note:_ Try to keep the code simple! In this course, we want to learn about the algorithms and we often do not need to use object-oriented design. 


## Task 1: Implement a simulation environment [2 Points]

The simple environment above is not very realistic. Your environment simulator needs to follow the PEAS description from above. It needs to:

* Initialize the environment by storing the state of each square (clean/dirty) and making some dirty.
* Keep track of the agent's position.
* Call the agent function repeatedly and provide the agent function with the sensor inputs.  
* React to the agent's actions. E.g, by removing dirt from a square or moving the agent around unless there is a wall in the way.
* Keep track of the performance measure. That is, track the agent's actions until all dirty squares are clean and count the number of actions it takes the agent to complete the task.

The easiest implementation for the environment is to hold an 2-dimensional array to represent if squares are clean or dirty and to call the agent function in a loop until all squares are clean or a predefined number of steps have been reached (i.e., the robot runs out of energy).

The simulation environment needs to work with the simple randomized agent program from above and then it can be used for your agent implementation in the tasks below.

In [5]:
import numpy as np

def environment(agent, n = 5, p = 0.2, x = random.choice(a=range(5)),
                y = random.choice(a=range(5)), max_steps = 100, verbose = True):
    
    ground = random.choice(a=[False, True], size=(n, n), p=[1-p, p])
    
    
    
    
    step = 0
        
    while np.count_nonzero(ground) > 0 and step < max_steps:
        bumpers = {
            'north': False,
            'south': False,
            'east': False,
            'west': False,
        }
            
        if x == 0:
            bumpers['north'] = True
        elif x == n - 1:
            bumpers['south'] = True
        
        if y == 0:
            bumpers['west'] = True
        elif y == n - 1:
            bumpers['east'] = True
            
        dirty = ground[x][y]
        action = agent(bumpers, dirty)
        if verbose: print("step", step ,"- position", x, ", ", y, "- action:", action)
        
        if action == 'suck':
            ground[x][y] = False
        elif action == 'north':
            x -= 1
        elif action == 'south':
            x += 1
        elif action == 'east':
            y += 1
        elif action == 'west':
            y -= 1
            
        if x > n - 1:
            x = n - 1
        elif x < 0:
            x = 0
            
        if y > n - 1:
            y = n - 1
        elif y < 0:
            y = 0
            
        step += 1
        
    return step

## Task 2:  Implement a simple reflex agent [1 Point] 

The simple reflex agent randomly walks around but reacts to the bumper sensor by not bumping into the wall and to dirt with sucking. Implement the agent program as a function.

_Note:_ The agent cannot directly use variable in the environment. It only gets the percepts as the arguments to the agent program function.

In [6]:
def simple_reflex_agent(bumpers, dirty):
    if dirty:
        return 'suck'
    
    actions = []
    
    for direction, bumper in bumpers.items():
        if not bumper:
            actions.append(direction)
            
    if actions:
        return random.choice(actions)
    else:
        return 'suck'

## Task 3: Implement a model-based reflex agent [3 Point]

This agent keeps track of the location and remembers where it has cleaned. Assume the agent knows how many squares the room has. It can move to a corner to determine its location and then is able to use more advanced navigation.

Describe how you define the __agent state__ and how your agent works before implementing it. _Note on implementing the state in Python:_ [Examples](https://github.com/mhahsler/CS7320-AI/blob/master/Python_Code_Examples/store_agent_state_information.ipynb)

The state of my model-based reflex agent is a bit more complex than simply tracking its position. The first component of the state is, however, position data, in the form of an *x* value, a *y* value, and *n*, which is the size of the *n* x *n* grid. Secondly, my implementation makes use of direction, which makes navigating the grid a bit simpler. The state also stores a *n* x *n* boolean array to track whether or not the agent has visited a square before. Lastly, the agent's state contains information about the nearest corner of the grid and whether or not it has arrived there yet. 

This last part of the state allows the agent to navigate to the corner of the grid without marking the squares it passes as "visited". While this is not exactly optimal, and was not my first attempt at a solution, it turns out that in trying to visit no squares twice, starting away from the corner of the grid can sometimes cause the agent to get "stuck" and not be able to find a path away from the previously visited squares.

The agent starts each action by cleaning the square if it is dirty. If the square is clean, and the agent has not yet navigated to the nearest corner, it will advance towards it. Once the agent has reached the corner, the agent then begins marking squares as "visited". The agent proceeds "forward" (initialized as 'north' or in the direction of negative *x*) until the next square in front of it has been visited or is out of the grid (which is checked by the bumpers). If the agent can not proceed forward, it "turns" right (adjusts the directions stored in the state to those right of each one) until it can. Once it can move forward, the agent returns the direction stored in the state as "forward" and updates its *x* and *y* coordinates.

This algorithm visually looks like the agent moving to the neareast corner and then visiting every square in a spiral towards the center, cleaning dirty squares as it passes over them. 

In [7]:
class ModelReflexAgent:
    def __init__(self, x, y, n):
        self.x = x
        self.y = y
        self.n = n
        
        self.forward = 'north'
        self.right   = 'east'
        self.back    = 'south'
        self.left    = 'west'
        
        self.visited = np.full((n, n), False)
        
        self.navigate_to_corner = True
        self.corner_x = (0 if x < (n / 2.0) else n - 1)
        self.corner_y = (0 if x < (n / 2.0) else n - 1)
        
    def act(self, bumpers, dirty):        
        if dirty:
            return 'suck'
        
        if self.navigate_to_corner:
            if self.x > self.corner_x:
                self.x -= 1
                return 'north'
            elif self.x < self.corner_x:
                self.x += 1
                return 'south'
            elif self.y < self.corner_y:
                self.y += 1
                return 'east'
            elif self.y > self.corner_y:
                self.y -= 1
                return 'west'
            else:
                self.navigate_to_corner = False

        self.visited[self.x][self.y] = True
        
        count = 0
        while not self.check(self.forward, bumpers):
            self.turn_right()
            count += 1
            # The agent is stuck if it turns right 4 times and still cannot advance
            # Return action 'suck' to keep it in place until it reaches max_steps
            if count > 4:
                return 'suck'
                            
        return self.advance()
        
    def check(self, direction, bumpers):
        if bumpers[direction]:
            return False
        elif direction == 'north':
            return not self.visited[self.x - 1][self.y]
        elif direction == 'south':
            return not self.visited[self.x + 1][self.y]
        elif direction == 'east':
            return not self.visited[self.x][self.y + 1]
        elif direction == 'west':
            return not self.visited[self.x][self.y - 1]

    def turn_right(self):
        self.forward, self.right, self.back, self.left = self.right, self.back, self.left, self.forward
        
    def advance(self):
        if self.forward == 'north':
            self.x = self.x - 1
        elif self.forward == 'south':
            self.x = self.x + 1
        elif self.forward == 'east':
            self.y = self.y + 1
        elif self.forward == 'west':
            self.y = self.y - 1
        return self.forward
        

## Task 4: Simulation study [3 Points]

Compare the performance (the performance measure is defined in the PEAS description above) of the agents using  environments of different size. E.g., $5 \times 5$, $10 \times 10$ and
$100 \times 100$. Use 100 random runs for each. Present the results in a suitable format (tables, graphs) and discuss the differences. 

Here is some help with [charts and tables.](https://github.com/mhahsler/CS7320-AI/blob/master/Python_Code_Examples/charts_and_tables.ipynb)

In [ ]:

avgs = {
    'rand': {},
    'rflx': {},
    'mbra': {}
}

trials = 100
sizes = [5, 10, 100]

for n in sizes:
    avgs['rand'][n] = {}
    avgs['rflx'][n] = {}
    avgs['mbra'][n] = {}
    avgs['rand'][n]['tot'] = 0
    avgs['rflx'][n]['tot'] = 0
    avgs['mbra'][n]['tot'] = 0
    

    for i in range(trials):
        x = random.choice(a=range(n))
        y = random.choice(a=range(n))
        model_reflex_agent = ModelReflexAgent(x, y, n).act
        
        avgs['rand'][n]['tot'] += environment(simple_randomized_agent,
                                              x = x, y = y, n = n,
                                              max_steps = 5 * (n**2), verbose = False)
        avgs['rflx'][n]['tot'] += environment(simple_reflex_agent,
                                              x = x, y = y, n = n,
                                              max_steps = 5 * (n**2), verbose = False)
        avgs['mbra'][n]['tot'] += environment(model_reflex_agent,
                                              x = x, y = y, n = n,
                                              max_steps = 5 * (n**2), verbose = False)
        
    avgs['rand'][n]['avg'] = avgs['rand'][n]['tot'] / trials
    avgs['rflx'][n]['avg'] = avgs['rflx'][n]['tot'] / trials
    avgs['mbra'][n]['avg'] = avgs['mbra'][n]['tot'] / trials

In [ ]:
avgs

Fill out the following table with the average performance measure for 100 random runs (you may also create this table with code):

| Size     | Randomized Agent | Simple Reflex Agent | Model-based Reflex Agent |
|----------|------------------|---------------------|--------------------------|
| 5x5     | 411.73 | 108.0 | 27.05 |
| 10x10   | 3006.74 | 881.64 | 122.75 |
| 100x100 | 846536.08 | 346545.62 | 12062.94 |

As expected, the model based agent far out performs the reflex based agent, increasingly so as the grid expands. In a 5x5 grid, the MBRA is nearly 4 times as fast as the simple reflex agent, but as the grid quadruples in size (25 to 100 squares), the factor increases to nearly 8 times. Once the grid reaches 10,000 squares, the simple reflex agent is over 28 times slower than the model based agent. I would expect the gap between the model based and simple reflex agent to grow even further when expanding the grid, due to randomness performing much worse as the grid size increases.

Another point of note is that, on average, the model based reflex agent seems to complete cleaning at a little over 1.2 times the number of squares in the grid., which fits very closely to what we would expect, given that it traverse the entire grid, but takes an extra step every time it cleans the approximately 20% of dirty squares. The other variation we likely see is the navigation to the corner introducing extra steps, and the completion of cleaning happening before a full traversal of the grid. I would wager that the larger the grid gets, the more this trend would continue towards a fixed value slightly above *1.2 x n<sup>2</sup>*.

## Task 5: Robustness of the agent implementations [1 Point] 

Describe how your agent implementations will perform 

* if it is put into a rectangular room with unknown size, 
* if the cleaning area can have an iregular shape (e.g., a hallway connecting two rooms), or 
* if the room contains obstacles (i.e., squares that it cannot pass through and trigger the bumper sensors).

### Answer
* My agent would, with a slight tweak to the corner navigation section of the algorithm, perform excellently in a room of unknown size. The only hiccup in its current iteration would be that it needs to know the size of the room n to determine which corner is nearest to its current position, but simply changing that code to navigate to 0,0 every time would fix that issue.
* My agent would be able to generally perform well in this environment with the same tweak mentioned above, however there definitely would be issues with some room configurations. Firstly, if there were an obstacle in the way of the agent's starting position and the top corner of the room, the agent may miss a signigicant portion of the room, or in other cases, such as a hallway one square wide, the agent might get itself stuck on one side of the room.
* A room that contains obstacles would likely provide a similar issue as the previous alteration, creating problems by forcing the agent to cut its own path off. The concept of visiting each square only once does not work perfectly for all shapes, but specifically works for an open square, so any variation on the shape of the environment (excluding simply scaling the environment) will result in similar issues.

## Graduate student advanced task: Obstacles [1 Point]

__Undergraduate students:__ This is a bonus task you can attempt if you like [+1 Bonus point].

1. Change your simulation environment tor run experiments for the following problem: Add random obstacle squares that also trigger the bumper sensor. The agent does not know where the obstacles are. Observe how this changes the performance of the three implementations.

2. Describe what would need to be done to perform better with obstacles. Add code if you can. 

In [8]:
import numpy as np

def obstacle_environment(agent, n = 5, p = 0.2, x = random.choice(a=range(5)),
                y = random.choice(a=range(5)), max_steps = 100, verbose = True):
    
    ground = random.choice(a=[False, True], size=(n, n), p=[1-p, p])
    
    
    
    
    step = 0
        
    while np.count_nonzero(ground) > 0 and step < max_steps:
        bumpers = {
            'north': False,
            'south': False,
            'east': False,
            'west': False,
        }
        
        ######################################################
        # Changes to add obstacles into environment ##########
        
        obstacles = np.full((n, n), False)
        possible = np.transpose(np.nonzero(np.invert(ground)))
        
        for i in possible:
            if random.choice(int(1/p)) == 0:
                obstacles[i[0]][i[1]] = True
        
        if (x == 0) or (obstacles[x - 1][y]):
            bumpers['north'] = True
        if (x == n) - 1 or (obstacles[x + 1][y]):
            bumpers['south'] = True
        if (y == 0) or (obstacles[x][y - 1]):
            bumpers['west'] = True
        if (y == n - 1) or (obstacles[x][y + 1]):
            bumpers['east'] = True
        
        ######################################################
        
        
        dirty = ground[x][y]
        action = agent(bumpers, dirty)
        if verbose: print("step", step ,"- position", x, ", ", y, "- action:", action)
        
        if action == 'suck':
            ground[x][y] = False
        elif action == 'north':
            x -= 1
        elif action == 'south':
            x += 1
        elif action == 'east':
            y += 1
        elif action == 'west':
            y -= 1
            
        if x > n - 1:
            x = n - 1
        elif x < 0:
            x = 0
            
        if y > n - 1:
            y = n - 1
        elif y < 0:
            y = 0
            
        step += 1
        
    return step

In [ ]:
obst_avgs = {
    'rand': {},
    'rflx': {},
    'mbra': {}
}

trials = 1
sizes = [5, 10, 100]

for n in sizes:
    obst_avgs['rand'][n] = {}
    obst_avgs['rflx'][n] = {}
    obst_avgs['mbra'][n] = {}
    obst_avgs['rand'][n]['tot'] = 0
    obst_avgs['rflx'][n]['tot'] = 0
    obst_avgs['mbra'][n]['tot'] = 0
    

    for i in range(trials):
        x = random.choice(a=range(n))
        y = random.choice(a=range(n))
        model_reflex_agent = ModelReflexAgent(x, y, n).act
        
        obst_avgs['rand'][n]['tot'] += obstacle_environment(simple_randomized_agent,
                                              x = x, y = y, n = n,
                                              max_steps = 5 * (n**2), verbose = True)
        obst_avgs['rflx'][n]['tot'] += obstacle_environment(simple_reflex_agent,
                                              x = x, y = y, n = n,
                                              max_steps = 5 * (n**2), verbose = True)
        obst_avgs['mbra'][n]['tot'] += obstacle_environment(model_reflex_agent,
                                              x = x, y = y, n = n,
                                              max_steps = 5 * (n**2), verbose = True)
        
    obst_avgs['rand'][n]['avg'] = obst_avgs['rand'][n]['tot'] / trials
    obst_avgs['rflx'][n]['avg'] = obst_avgs['rflx'][n]['tot'] / trials
    obst_avgs['mbra'][n]['avg'] = obst_avgs['mbra'][n]['tot'] / trials

step 0 - position 1 ,  4 - action: north
step 1 - position 0 ,  4 - action: north
step 2 - position 0 ,  4 - action: east
step 3 - position 0 ,  4 - action: south
step 4 - position 1 ,  4 - action: suck
step 5 - position 1 ,  4 - action: west
step 6 - position 1 ,  3 - action: north
step 7 - position 0 ,  3 - action: west
step 8 - position 0 ,  2 - action: north
step 9 - position 0 ,  2 - action: east
step 10 - position 0 ,  3 - action: west
step 11 - position 0 ,  2 - action: east
step 12 - position 0 ,  3 - action: suck
step 13 - position 0 ,  3 - action: west
step 14 - position 0 ,  2 - action: suck
step 15 - position 0 ,  2 - action: west
step 16 - position 0 ,  1 - action: south
step 17 - position 1 ,  1 - action: west
step 18 - position 1 ,  0 - action: north
step 19 - position 0 ,  0 - action: north
step 20 - position 0 ,  0 - action: suck
step 21 - position 0 ,  0 - action: west
step 22 - position 0 ,  0 - action: east
step 23 - position 0 ,  1 - action: south
step 24 - positio

step 33 - position 0 ,  4 - action: suck
step 34 - position 0 ,  4 - action: suck
step 35 - position 0 ,  4 - action: suck
step 36 - position 0 ,  4 - action: suck
step 37 - position 0 ,  4 - action: suck
step 38 - position 0 ,  4 - action: suck
step 39 - position 0 ,  4 - action: suck
step 40 - position 0 ,  4 - action: suck
step 41 - position 0 ,  4 - action: suck
step 42 - position 0 ,  4 - action: suck
step 43 - position 0 ,  4 - action: suck
step 44 - position 0 ,  4 - action: suck
step 45 - position 0 ,  4 - action: suck
step 46 - position 0 ,  4 - action: suck
step 47 - position 0 ,  4 - action: suck
step 48 - position 0 ,  4 - action: suck
step 49 - position 0 ,  4 - action: suck
step 50 - position 0 ,  4 - action: suck
step 51 - position 0 ,  4 - action: suck
step 52 - position 0 ,  4 - action: suck
step 53 - position 0 ,  4 - action: suck
step 54 - position 0 ,  4 - action: suck
step 55 - position 0 ,  4 - action: suck
step 56 - position 0 ,  4 - action: suck
step 57 - positi

step 117 - position 4 ,  9 - action: south
step 118 - position 5 ,  9 - action: south
step 119 - position 6 ,  9 - action: east
step 120 - position 6 ,  9 - action: north
step 121 - position 5 ,  9 - action: west
step 122 - position 5 ,  8 - action: west
step 123 - position 5 ,  7 - action: east
step 124 - position 5 ,  8 - action: west
step 125 - position 5 ,  7 - action: east
step 126 - position 5 ,  8 - action: west
step 127 - position 5 ,  7 - action: east
step 128 - position 5 ,  8 - action: south
step 129 - position 6 ,  8 - action: west
step 130 - position 6 ,  7 - action: north
step 131 - position 5 ,  7 - action: east
step 132 - position 5 ,  8 - action: suck
step 133 - position 5 ,  8 - action: suck
step 134 - position 5 ,  8 - action: west
step 135 - position 5 ,  7 - action: north
step 136 - position 4 ,  7 - action: north
step 137 - position 3 ,  7 - action: west
step 138 - position 3 ,  6 - action: north
step 139 - position 2 ,  6 - action: east
step 140 - position 2 ,  7

step 319 - position 0 ,  2 - action: east
step 320 - position 0 ,  3 - action: east
step 321 - position 0 ,  4 - action: south
step 322 - position 1 ,  4 - action: east
step 323 - position 1 ,  5 - action: north
step 324 - position 0 ,  5 - action: south
step 325 - position 1 ,  5 - action: south
step 326 - position 2 ,  5 - action: west
step 327 - position 2 ,  4 - action: north
step 328 - position 1 ,  4 - action: north
step 329 - position 0 ,  4 - action: suck
step 330 - position 0 ,  4 - action: east
step 331 - position 0 ,  5 - action: east
step 332 - position 0 ,  6 - action: west
step 333 - position 0 ,  5 - action: west
step 334 - position 0 ,  4 - action: north
step 335 - position 0 ,  4 - action: west
step 336 - position 0 ,  3 - action: north
step 337 - position 0 ,  3 - action: west
step 338 - position 0 ,  2 - action: north
step 339 - position 0 ,  2 - action: south
step 340 - position 1 ,  2 - action: west
step 341 - position 1 ,  1 - action: west
step 342 - position 1 , 

step 23 - position 0 ,  2 - action: east
step 24 - position 0 ,  3 - action: west
step 25 - position 0 ,  2 - action: east
step 26 - position 0 ,  3 - action: east
step 27 - position 0 ,  4 - action: east
step 28 - position 0 ,  5 - action: east
step 29 - position 0 ,  6 - action: west
step 30 - position 0 ,  5 - action: east
step 31 - position 0 ,  6 - action: west
step 32 - position 0 ,  5 - action: west
step 33 - position 0 ,  4 - action: east
step 34 - position 0 ,  5 - action: west
step 35 - position 0 ,  4 - action: east
step 36 - position 0 ,  5 - action: east
step 37 - position 0 ,  6 - action: east
step 38 - position 0 ,  7 - action: west
step 39 - position 0 ,  6 - action: west
step 40 - position 0 ,  5 - action: west
step 41 - position 0 ,  4 - action: east
step 42 - position 0 ,  5 - action: west
step 43 - position 0 ,  4 - action: west
step 44 - position 0 ,  3 - action: east
step 45 - position 0 ,  4 - action: east
step 46 - position 0 ,  5 - action: west
step 47 - positi

step 270 - position 0 ,  2 - action: west
step 271 - position 0 ,  1 - action: east
step 272 - position 0 ,  2 - action: east
step 273 - position 0 ,  3 - action: west
step 274 - position 0 ,  2 - action: east
step 275 - position 0 ,  3 - action: east
step 276 - position 0 ,  4 - action: west
step 277 - position 0 ,  3 - action: west
step 278 - position 0 ,  2 - action: east
step 279 - position 0 ,  3 - action: west
step 280 - position 0 ,  2 - action: west
step 281 - position 0 ,  1 - action: west
step 282 - position 0 ,  0 - action: east
step 283 - position 0 ,  1 - action: east
step 284 - position 0 ,  2 - action: east
step 285 - position 0 ,  3 - action: east
step 286 - position 0 ,  4 - action: east
step 287 - position 0 ,  5 - action: east
step 288 - position 0 ,  6 - action: west
step 289 - position 0 ,  5 - action: suck
step 290 - position 0 ,  5 - action: west
step 291 - position 0 ,  4 - action: west
step 292 - position 0 ,  3 - action: east
step 293 - position 0 ,  4 - actio

step 466 - position 0 ,  8 - action: east
step 467 - position 0 ,  9 - action: west
step 468 - position 0 ,  8 - action: west
step 469 - position 0 ,  7 - action: west
step 470 - position 0 ,  6 - action: west
step 471 - position 0 ,  5 - action: east
step 472 - position 0 ,  6 - action: east
step 473 - position 0 ,  7 - action: west
step 474 - position 0 ,  6 - action: east
step 475 - position 0 ,  7 - action: west
step 476 - position 0 ,  6 - action: west
step 477 - position 0 ,  5 - action: east
step 478 - position 0 ,  6 - action: west
step 479 - position 0 ,  5 - action: west
step 480 - position 0 ,  4 - action: east
step 481 - position 0 ,  5 - action: east
step 482 - position 0 ,  6 - action: west
step 483 - position 0 ,  5 - action: east
step 484 - position 0 ,  6 - action: west
step 485 - position 0 ,  5 - action: east
step 486 - position 0 ,  6 - action: west
step 487 - position 0 ,  5 - action: west
step 488 - position 0 ,  4 - action: east
step 489 - position 0 ,  5 - actio

step 184 - position 0 ,  9 - action: suck
step 185 - position 0 ,  9 - action: suck
step 186 - position 0 ,  9 - action: suck
step 187 - position 0 ,  9 - action: suck
step 188 - position 0 ,  9 - action: suck
step 189 - position 0 ,  9 - action: suck
step 190 - position 0 ,  9 - action: suck
step 191 - position 0 ,  9 - action: suck
step 192 - position 0 ,  9 - action: suck
step 193 - position 0 ,  9 - action: suck
step 194 - position 0 ,  9 - action: suck
step 195 - position 0 ,  9 - action: suck
step 196 - position 0 ,  9 - action: suck
step 197 - position 0 ,  9 - action: suck
step 198 - position 0 ,  9 - action: suck
step 199 - position 0 ,  9 - action: suck
step 200 - position 0 ,  9 - action: suck
step 201 - position 0 ,  9 - action: suck
step 202 - position 0 ,  9 - action: suck
step 203 - position 0 ,  9 - action: suck
step 204 - position 0 ,  9 - action: suck
step 205 - position 0 ,  9 - action: suck
step 206 - position 0 ,  9 - action: suck
step 207 - position 0 ,  9 - actio

step 394 - position 0 ,  9 - action: suck
step 395 - position 0 ,  9 - action: suck
step 396 - position 0 ,  9 - action: suck
step 397 - position 0 ,  9 - action: suck
step 398 - position 0 ,  9 - action: suck
step 399 - position 0 ,  9 - action: suck
step 400 - position 0 ,  9 - action: suck
step 401 - position 0 ,  9 - action: suck
step 402 - position 0 ,  9 - action: suck
step 403 - position 0 ,  9 - action: suck
step 404 - position 0 ,  9 - action: suck
step 405 - position 0 ,  9 - action: suck
step 406 - position 0 ,  9 - action: suck
step 407 - position 0 ,  9 - action: suck
step 408 - position 0 ,  9 - action: suck
step 409 - position 0 ,  9 - action: suck
step 410 - position 0 ,  9 - action: suck
step 411 - position 0 ,  9 - action: suck
step 412 - position 0 ,  9 - action: suck
step 413 - position 0 ,  9 - action: suck
step 414 - position 0 ,  9 - action: suck
step 415 - position 0 ,  9 - action: suck
step 416 - position 0 ,  9 - action: suck
step 417 - position 0 ,  9 - actio

step 91 - position 0 ,  42 - action: west
step 92 - position 0 ,  41 - action: east
step 93 - position 0 ,  42 - action: north
step 94 - position 0 ,  42 - action: west
step 95 - position 0 ,  41 - action: west
step 96 - position 0 ,  40 - action: south
step 97 - position 1 ,  40 - action: west
step 98 - position 1 ,  39 - action: north
step 99 - position 0 ,  39 - action: north
step 100 - position 0 ,  39 - action: east
step 101 - position 0 ,  40 - action: suck
step 102 - position 0 ,  40 - action: south
step 103 - position 1 ,  40 - action: north
step 104 - position 0 ,  40 - action: east
step 105 - position 0 ,  41 - action: north
step 106 - position 0 ,  41 - action: north
step 107 - position 0 ,  41 - action: east
step 108 - position 0 ,  42 - action: north
step 109 - position 0 ,  42 - action: west
step 110 - position 0 ,  41 - action: suck
step 111 - position 0 ,  41 - action: suck
step 112 - position 0 ,  41 - action: east
step 113 - position 0 ,  42 - action: east
step 114 - 

In [ ]:
obst_avgs

## More advanced tasks to think about

You can think about these:

* __Unknown environment with obstacles:__ Implement an agent for an environment where the agent does not know how large the environment is (we assume it is rectangular), where it starts or where the obstacles are. An option would be to always move to the closest unchecked/uncleaned square.

* __Utility-based agent:__ Change the environment, so each square has a fixed probability of getting dirty again. We assume the agent has learned this information over time. For the implementation, we give this information to the agent as a 2-dimensional array of probabilities  Cleaning one dirty square produces a utility of 1. Implement a utility-based agent that maximizes the expected utility over one full charge which lasts for 10000 time steps. This is very tricky!

In [ ]:
# Your ideas/code